OBSOLETE NOTEBOOK. REMOVED IN THE FUTURE
===================

In [116]:
from transformers import BertModel, BertConfig
from transformers import BertForMaskedLM
from transformers import BertTokenizer
from transformers import AdamW
from transformers import Trainer, TrainingArguments

import torch
import numpy as np
import spacy
import re

from datetime import datetime
from tqdm import tqdm

In [2]:
# pos_list = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
# ner_list = ['PERSON', 'NORP', 'FAC', 'ORG', 'GPE', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART', 'LAW', 'LANGUAGE', 'DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'ORDINAL', 'CARDINAL']

In [3]:
sentences = ["When on board H.M.S. 'Beagle,' as naturalist, I was much struck with certain facts in the distribution of the inhabitants of South America, and in the geological relations of the present to the past inhabitants of that continent.",
 'These facts seemed to me to throw some light on the origin of species--that mystery of mysteries, as it has been called by one of our greatest philosophers.',
 'On my return home, it occurred to me, in 1837, that something might perhaps be made out on this question by patiently accumulating and reflecting on all sorts of facts which could possibly have any bearing on it.',
 "After five years' work I allowed myself to speculate on the subject, and drew up some short notes; these I enlarged in 1844 into a sketch of the conclusions, which then seemed to me probable: from that period to the present day I have steadily pursued the same object.",
 'I hope that I may be excused for entering on these personal details, as I give them to show that I have not been hasty in coming to a decision.']

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
nlp = spacy.load("en_core_web_sm")

In [5]:
tokenizer.model_max_length

512

In [6]:
def whole_word_MO_tokenization_and_masking(tokenizer, nlp_model, sequence: str):
        """
        posoi: Part-Of-Speech of interest
        
        Performs whole-word-masking based on selected posoi.
        
        POS possibilities:['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 
                            'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
                             
        TODO: What if no tokens are masked?
        
        """
        print('loading:', datetime.now().time())
        spacy_sentence = nlp_model(sequence, disable=["parser"])
        
        POS_list = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 
                            'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
        NER_list = ['PERSON', 'NORP', 'FAC', 'ORG', 'GPE', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART', 
                    'LAW', 'LANGUAGE', 'DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'ORDINAL', 'CARDINAL']
        NER_pairs = ['']
        
        input_ids = tokenizer.encode(sequence, add_special_tokens=False)
        input_tokens = tokenizer.convert_ids_to_tokens(input_ids)
        print(sequence)
        print(input_tokens)
        sequence_pos_list = [token.pos_ for token in spacy_sentence]
        sequence_pos_frequency = {pos: sequence_pos_list.count(pos) for pos in sequence_pos_list}
        
        modified_input_list = []
        
        #POS-masking
        print('pos-start:', datetime.now().time())
        for posoi in sequence_pos_frequency.keys():
            posoi_vocab = [token.text.lower() for token in spacy_sentence if token.pos_ == posoi]
            
            mask_indices = []
            composite_word_indices = []
            composite_word_tokens = []
            for (i, token) in enumerate(input_tokens):
                if token == "[CLS]" or token == "[SEP]":
                    continue
                elif token.startswith("##"):
                    composite_word_indices.append(i)
                    composite_word_tokens.append(token)
                    print("".join([x.strip("##") for x in composite_word_tokens]))
                    if "".join([x.strip("##") for x in composite_word_tokens]) in posoi_vocab:
                        mask_indices = mask_indices + composite_word_indices

                elif token in posoi_vocab:
                    mask_indices.append(i)
                else:
                    composite_word_indices = [i]
                    composite_word_tokens = [token]

            mask_labels = [1 if i in mask_indices else 0 for i in range(len(input_tokens))]
            masked_tokens = [x if mask_labels[i] == 0 else 103 for i,x in enumerate(input_ids)]
            masked_input = tokenizer.decode(masked_tokens)         
            modified_input_list.append(masked_input)

        #POS-based lemmatization
        replacement_tuples = [(token.text, token.lemma_) for token in spacy_sentence if token.text.lower() != token.lemma_]
        #print(replacement_tuples)
        pos_replaced_sentence = sequence
        for replacement in replacement_tuples:
            pos_replaced_sentence = re.sub(r'\b' + replacement[0] + r'\b', replacement[1], pos_replaced_sentence)

        pos_replaced_sentence = pos_replaced_sentence.replace("  ", " ")
        print('Lemma', pos_replaced_sentence)
        modified_input_list.append(pos_replaced_sentence)
        
        #NER-based swapping of time-place (if present)
        print('ner-start:', datetime.now().time())
        ner_swapped_sentence = spacy_sentence.text
        for ent in spacy_sentence.ents:
            if ent.label_ == 'TIME':
                time_substring = ner_swapped_sentence[ent.start_char:ent.end_char].split(" ")
                time_substring.reverse()
                ner_swapped_sentence = ner_swapped_sentence.replace(ner_swapped_sentence[ent.start_char:ent.end_char], " ".join(time_substring))
        print('NER', ner_swapped_sentence)
        modified_input_list.append(ner_swapped_sentence)
        
        
        #TODO future ideas
        #
        #
        
    
        #actually tokenize input
        inputs = tokenizer(modified_input_list, return_tensors="pt", padding=True)

        inputs['labels'] = tokenizer([sequence for i in range(0,inputs['input_ids'].shape[0])], 
                                     return_attention_mask=False, 
                                     return_token_type_ids=False,
                                     return_tensors='pt', padding=True)['input_ids']
        
        return inputs

In [7]:
print(datetime.now().time())
test_sentence = "Anne went to the Albert Heijn at 5 o'clock to buy some milk for me."
example_sentence_inputs = whole_word_MO_tokenization_and_masking(tokenizer=tokenizer, nlp_model=nlp, sequence=test_sentence)
print(datetime.now().time())

16:35:24.390461
loading: 16:35:24.391457
Anne went to the Albert Heijn at 5 o'clock to buy some milk for me.
['anne', 'went', 'to', 'the', 'albert', 'he', '##ij', '##n', 'at', '5', 'o', "'", 'clock', 'to', 'buy', 'some', 'milk', 'for', 'me', '.']
pos-start: 16:35:24.401430
heij
heijn
heij
heijn
heij
heijn
heij
heijn
heij
heijn
heij
heijn
heij
heijn
heij
heijn
heij
heijn
Lemma Anne go to the Albert Heijn at 5 o'clock to buy some milk for I.
ner-start: 16:35:24.403425
NER Anne went to the Albert Heijn at o'clock 5 to buy some milk for me.
16:35:24.413399


In [8]:
spacy_sentence = nlp("Anne went to the Albert Heijn at 5 o'clock to buy some milk for me.")

for token in spacy_sentence:
    print(token.text, token.pos_)


Anne PROPN
went VERB
to ADP
the DET
Albert PROPN
Heijn PROPN
at ADP
5 NUM
o'clock NOUN
to PART
buy VERB
some DET
milk NOUN
for ADP
me PRON
. PUNCT


In [9]:
example_sentence_inputs

{'input_ids': tensor([[  101,   103,  2253,  2000,  1996,   103,   103,   103,   103,  2012,
          1019,  1051,  1005,  5119,  2000,  4965,  2070,  6501,  2005,  2033,
          1012,   102],
        [  101,  4776,   103,  2000,  1996,  4789,  2002, 28418,  2078,  2012,
          1019,  1051,  1005,  5119,  2000,   103,  2070,  6501,  2005,  2033,
          1012,   102],
        [  101,  4776,  2253,   103,  1996,  4789,  2002, 28418,  2078,   103,
          1019,  1051,  1005,  5119,   103,  4965,  2070,  6501,   103,  2033,
          1012,   102],
        [  101,  4776,  2253,  2000,   103,  4789,  2002, 28418,  2078,  2012,
          1019,  1051,  1005,  5119,  2000,  4965,   103,  6501,  2005,  2033,
          1012,   102],
        [  101,  4776,  2253,  2000,  1996,  4789,  2002, 28418,  2078,  2012,
           103,  1051,  1005,  5119,  2000,  4965,  2070,  6501,  2005,  2033,
          1012,   102],
        [  101,  4776,  2253,  2000,  1996,  4789,  2002, 28418,  2078,  201

In [10]:
example_sentence_inputs['input_ids']

tensor([[  101,   103,  2253,  2000,  1996,   103,   103,   103,   103,  2012,
          1019,  1051,  1005,  5119,  2000,  4965,  2070,  6501,  2005,  2033,
          1012,   102],
        [  101,  4776,   103,  2000,  1996,  4789,  2002, 28418,  2078,  2012,
          1019,  1051,  1005,  5119,  2000,   103,  2070,  6501,  2005,  2033,
          1012,   102],
        [  101,  4776,  2253,   103,  1996,  4789,  2002, 28418,  2078,   103,
          1019,  1051,  1005,  5119,   103,  4965,  2070,  6501,   103,  2033,
          1012,   102],
        [  101,  4776,  2253,  2000,   103,  4789,  2002, 28418,  2078,  2012,
          1019,  1051,  1005,  5119,  2000,  4965,   103,  6501,  2005,  2033,
          1012,   102],
        [  101,  4776,  2253,  2000,  1996,  4789,  2002, 28418,  2078,  2012,
           103,  1051,  1005,  5119,  2000,  4965,  2070,  6501,  2005,  2033,
          1012,   102],
        [  101,  4776,  2253,  2000,  1996,  4789,  2002, 28418,  2078,  2012,
          1

In [11]:
text= 'On their very first meeting, Gilbert had not been pleasantly impressed with Hardy. But he soon saw that the man had a certain rugged strength, and there was no doubt he had suffered from the depredations of Mexico\'s casual visitors, and was ready to protect not only his own interests but those of any newcomers. He seemed to have the spirit of fair-mindedness; and he believed firmly in the possibilities of this magic land, particularly for young men. "It\'s God\'s country," he told Gilbert on more than one occasion. "Get into the soil all you can. Dig--and dig deep."'

In [12]:
whole_word_MO_tokenization_and_masking(tokenizer=tokenizer, nlp_model=nlp, sequence=text)

loading: 16:35:24.483213
On their very first meeting, Gilbert had not been pleasantly impressed with Hardy. But he soon saw that the man had a certain rugged strength, and there was no doubt he had suffered from the depredations of Mexico's casual visitors, and was ready to protect not only his own interests but those of any newcomers. He seemed to have the spirit of fair-mindedness; and he believed firmly in the possibilities of this magic land, particularly for young men. "It's God's country," he told Gilbert on more than one occasion. "Get into the soil all you can. Dig--and dig deep."
['on', 'their', 'very', 'first', 'meeting', ',', 'gilbert', 'had', 'not', 'been', 'pleasantly', 'impressed', 'with', 'hardy', '.', 'but', 'he', 'soon', 'saw', 'that', 'the', 'man', 'had', 'a', 'certain', 'rugged', 'strength', ',', 'and', 'there', 'was', 'no', 'doubt', 'he', 'had', 'suffered', 'from', 'the', 'de', '##pre', '##dation', '##s', 'of', 'mexico', "'", 's', 'casual', 'visitors', ',', 'and', '

{'input_ids': tensor([[ 101,  103, 2037,  ..., 1012, 1000,  102],
        [ 101, 2006,  103,  ..., 1012, 1000,  102],
        [ 101, 2006, 2037,  ..., 1012, 1000,  102],
        ...,
        [ 101, 2006, 2037,  ..., 1012, 1000,  102],
        [ 101, 2006, 2037,  ...,    0,    0,    0],
        [ 101, 2006, 2037,  ..., 1012, 1000,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[ 101, 2006, 2037,  ..., 1012, 1000,  102],
        [ 101, 2006, 2037,  ..., 1012, 1000,  102],
        [ 101, 2006, 2037,  ..., 1012, 1000,  102],
        ...,
      

In [13]:
text2 = '"Sturgis telegraphed me that there was a big possibility of a new vein of oil down on the border," Pell was telling her. "Some important men want to talk things over with me at Bisbee.I want to get started in a day or two.Don\'t take your maid.It\'s a rough country, but you\'ll be all right.Just old clothes.You can ride a lot, so bring your habit.I\'ll be busy most of the time; but I think you\'ll like the trip.Never been down that way, have you?"'
whole_word_MO_tokenization_and_masking(tokenizer=tokenizer, nlp_model=nlp, sequence=text2)

loading: 16:35:24.608878
"Sturgis telegraphed me that there was a big possibility of a new vein of oil down on the border," Pell was telling her. "Some important men want to talk things over with me at Bisbee.I want to get started in a day or two.Don't take your maid.It's a rough country, but you'll be all right.Just old clothes.You can ride a lot, so bring your habit.I'll be busy most of the time; but I think you'll like the trip.Never been down that way, have you?"
['"', 'stu', '##rg', '##is', 'telegraph', '##ed', 'me', 'that', 'there', 'was', 'a', 'big', 'possibility', 'of', 'a', 'new', 'vein', 'of', 'oil', 'down', 'on', 'the', 'border', ',', '"', 'pe', '##ll', 'was', 'telling', 'her', '.', '"', 'some', 'important', 'men', 'want', 'to', 'talk', 'things', 'over', 'with', 'me', 'at', 'bis', '##bee', '.', 'i', 'want', 'to', 'get', 'started', 'in', 'a', 'day', 'or', 'two', '.', 'don', "'", 't', 'take', 'your', 'maid', '.', 'it', "'", 's', 'a', 'rough', 'country', ',', 'but', 'you', "'",

{'input_ids': tensor([[  101,   103, 24646,  ...,   103,   103,   102],
        [  101,  1000,   103,  ...,  1029,  1000,   102],
        [  101,  1000, 24646,  ...,  1029,  1000,   102],
        ...,
        [  101,  1000, 24646,  ...,  1029,  1000,   102],
        [  101,  1000, 24646,  ...,  1000,   102,     0],
        [  101,  1000, 24646,  ...,  1029,  1000,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[  101,  1000, 24646,  ...,  1029,  1000,   102],
        [  101,  1000, 24646,  ...,  1029,  1000,   102],
        [  101,  1000, 

In [14]:
bert_tiny_config = {"hidden_size": 128, 
                    "hidden_act": "gelu", 
                    "initializer_range": 0.02, 
                    "vocab_size": 30522, 
                    "hidden_dropout_prob": 0.1, 
                    "num_attention_heads": 2, 
                    "type_vocab_size": 2, 
                    "max_position_embeddings": 512, 
                    "num_hidden_layers": 2, 
                    "intermediate_size": 512, 
                    "attention_probs_dropout_prob": 0.1}


model = BertForMaskedLM(config=BertConfig(**bert_tiny_config))
model.train()
optimizer = AdamW(model.parameters(), lr=1e-5)

In [15]:
outputs = model(**example_sentence_inputs, return_dict=True)

In [16]:
outputs

MaskedLMOutput(loss=tensor(10.2835, grad_fn=<NllLossBackward>), logits=tensor([[[ 0.0200,  0.1465, -0.3637,  ...,  0.0395, -0.0656, -0.3321],
         [ 0.0484, -0.2371, -0.4776,  ...,  0.1257,  0.0007, -0.1336],
         [ 0.1460,  0.0925, -0.4497,  ..., -0.2280, -0.1593, -0.1718],
         ...,
         [-0.0520,  0.1370, -0.1459,  ..., -0.1432,  0.1006, -0.1996],
         [-0.4075, -0.0481, -0.0996,  ..., -0.4267,  0.1172,  0.0081],
         [ 0.1919,  0.5225, -0.4625,  ..., -0.0241,  0.0064, -0.0537]],

        [[ 0.0859,  0.0773, -0.3543,  ...,  0.0837,  0.0407, -0.4331],
         [-0.1709, -0.2265, -0.0413,  ..., -0.2139,  0.2773, -0.3632],
         [ 0.1152,  0.0283, -0.6541,  ...,  0.0077, -0.0244,  0.0622],
         ...,
         [-0.0490,  0.2417, -0.0900,  ..., -0.1173,  0.0537, -0.1346],
         [-0.1855, -0.0206, -0.2053,  ..., -0.6487,  0.2193, -0.1390],
         [ 0.2551,  0.5884, -0.4756,  ..., -0.0008,  0.0868, -0.0447]],

        [[ 0.0328,  0.0431, -0.3068,  ...,  0

In [17]:
loss = outputs.loss

In [18]:
loss.backward()

In [19]:
optimizer.step()

In [20]:
model(**example_sentence_inputs, return_dict=True)

MaskedLMOutput(loss=tensor(10.2790, grad_fn=<NllLossBackward>), logits=tensor([[[-5.9162e-02,  2.9790e-01, -4.1157e-01,  ...,  1.2211e-01,
          -5.7689e-02, -4.1296e-01],
         [ 5.6311e-02, -1.5754e-01, -4.6341e-01,  ...,  3.0909e-01,
           1.1902e-02, -1.3802e-01],
         [ 1.4994e-01,  1.5642e-01, -4.9551e-01,  ..., -1.0557e-01,
          -1.2618e-01, -4.2685e-01],
         ...,
         [-4.0045e-01,  1.8661e-01,  6.2309e-02,  ..., -3.6598e-01,
           2.0491e-01, -2.4264e-02],
         [-2.5544e-01, -2.8214e-02, -2.7622e-01,  ..., -4.7163e-01,
           1.0190e-01, -4.8512e-02],
         [ 2.6299e-01,  5.7057e-01, -5.0931e-01,  ..., -9.6850e-02,
          -8.2659e-02, -1.0471e-02]],

        [[-1.4571e-01,  2.3586e-01, -4.0504e-01,  ...,  2.0158e-01,
           1.2172e-01, -3.0752e-01],
         [-1.4247e-01, -1.9875e-01, -1.2193e-02,  ..., -1.7029e-01,
           3.3950e-01, -3.0286e-01],
         [ 2.2047e-02,  6.3670e-02, -5.4502e-01,  ..., -1.1023e-01,
     

In [21]:
class MODataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = {key: val for key, val in encodings.items() if key != 'labels'}
        self.labels = encodings['labels']

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MODataset(example_sentence_inputs)
train_dataset

In [22]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=256,  # batch size per device during training
    per_device_eval_batch_size=256,   # batch size for evaluation
    learning_rate=1e-5,     
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=None            # evaluation dataset
)

In [23]:
trainer.train()

TrainOutput(global_step=3, training_loss=10.275548299153646)

Custom tokenizer
=====================

In [24]:
from tokenizer.tokenizer import StrategizedTokenizer

In [25]:
ST_tokenizer = StrategizedTokenizer()

In [26]:
inputs = ST_tokenizer.tokenize("Anne went to the Albert Heijn at 5 o'clock to buy some milk for me.")
inputs

{'input_ids': tensor([[ 101,  103, 2253,  ...,    0,    0,    0],
        [ 101, 4776,  103,  ...,    0,    0,    0],
        [ 101, 4776, 2253,  ...,    0,    0,    0],
        ...,
        [ 101, 4776, 2253,  ...,    0,    0,    0],
        [ 101, 4776, 2175,  ...,    0,    0,    0],
        [ 101, 4776, 2253,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 101, 4776, 2253,  ...,    0,    0,    0],
        [ 101, 4776, 2253,  ...,    0,    0,    0],
        [ 101, 4776, 2253,  ...,    0,    0,    0],
        ...,
        [ 101, 4776, 2253,  ...,    0,    0,    0],
        [ 101, 4776, 2253,  ...,    0,    0,    0],
        [ 101, 4776, 2253,  ...,    0,    0,    0]])}

In [3]:
import os
os.environ.get()

TypeError: get() missing 1 required positional argument: 'key'

In [27]:
from cleaner_utils import super_cleaner
from pretraining_data_utils import book_properties, make_df_book_properties
from gutenberg.acquire import load_etext
from tokenizer.tokenizer import StrategizedTokenizer

In [28]:
sentences = super_cleaner(load_etext(16968), -1, verify_deletions=False)

In [29]:
bert_tiny_config = {"hidden_size": 128, 
                    "hidden_act": "gelu", 
                    "initializer_range": 0.02, 
                    "vocab_size": 30522, 
                    "hidden_dropout_prob": 0.1, 
                    "num_attention_heads": 2, 
                    "type_vocab_size": 2, 
                    "max_position_embeddings": 512, 
                    "num_hidden_layers": 2, 
                    "intermediate_size": 512, 
                    "attention_probs_dropout_prob": 0.1}

model = BertForMaskedLM(config=BertConfig(**bert_tiny_config))

In [50]:
ds.encodings['input_ids'].dtype

torch.float32

In [65]:
#Size in KB
#Number of entries
#Tensor size
#2 tensors, input_ids and labels
#4 bytes
#divide by 1024 to go to kilobytes

def compute_filesize(entries, tensor_size):
    return '{} KB'.format(entries*tensor_size*2*4/1024), '{} MB'.format(entries*tensor_size*2*4/(1024*1024))

compute_filesize(22449, 512)

('89796.0 KB', '87.69140625 MB')

In [93]:
compute_filesize(1e5, 512)

('400000.0 KB', '390.625 MB')

In [94]:
compute_filesize(1e6, 512)

('4000000.0 KB', '3906.25 MB')

In [70]:
def memory_footprint(parameters):
    return '{} KB'.format(parameters*4/1024), '{} MB'.format(parameters*4/(1024*1024)), '{} GB'.format(parameters*4/(1024*1024*1024))

In [79]:
'BERT-tiny: ', memory_footprint(4.4 * 1e6)

('BERT-tiny: ', ('17187.5 KB', '16.78466796875 MB', '0.016391277313232422 GB'))

In [80]:
'BERT-base: ', memory_footprint(110.1 *1e6)

('BERT-base: ',
 ('430078.125 KB', '419.9981689453125 MB', '0.41015446186065674 GB'))

In [81]:
'GPT3 ', memory_footprint(175 *1e9)

('GPT3 ', ('683593750.0 KB', '667572.021484375 MB', '651.925802230835 GB'))

Testing some classification model
====================

In [ ]:
from transformers import AdamW
# From paper:
# lr: 1e-4
# Beta1 = 0.9 (default)
# Beta2 = 0.999 (default)
# L2 weight decay = 0.01

# Longer sequences are disproportionately expensive
# because attention is quadratic to the sequence
# length. To speed up pretraing in our experiments,
# we pre-train the model with sequence length of
# 128 for 90% of the steps. Then, we train the rest
# 10% of the steps of sequence of 512 to learn the
# positional embeddings.



#Batch size 256 for 1e6 steps


optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

In [ ]:
sentences[0]

In [ ]:
#doc = nlp("That's a lot better. He was finally walking to the beaches. There he had a meeting with his father. Afterwards, he read a book. The fishing rod that he used was really old")
doc = nlp("When on board H.M.S. 'Beagle,' as naturalist, I was much struck with certain facts in the distribution of the inhabitants of South America, and in the geological relations of the present to the past inhabitants of that continent.")

for token in doc:
    print(token.text, token.pos_, token.lemma_)

In [ ]:
doc = nlp("San Francisco is a long drive away from here. Ah, I forgot what I was doing. He had to get a new pair of shoes.")

for token in doc:
    print(token.text, token.pos_, token.lemma_)